In [ ]:
#clone and install dependencies
!pip install xformers
!pip install bitsandbytes
!pip uninstall torch torchvision -y

!git clone https://github.com/huggingface/diffusers 
!cd diffusers && pip install . && cd examples/dreambooth && pip install -r requirements.txt

!pip install --upgrade peft
!pip install wandb # for outputting images mid training

In [ ]:
#create environment
!accelerate config default

import os
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from transformers import CLIPProcessor, CLIPModel
import torch
import torchvision.transforms as T
import torch.nn.functional as F
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import timm
import numpy as np

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
from huggingface_hub import login

# Use your Hugging Face token for login
login(token="YOUR_HUGGING_FACE_TOKEN")

os.environ["WANDB_MODE"] = "online"  
# Replace with your W&B API key
os.environ["WANDB_API_KEY"] = "YOUR_WANDB_KEY"  

HF_ACCOUNT = "YOUR_HF_ACCOUNT_NAME"

In [ ]:
new_token = "[V]"

In [ ]:
#get model
pretrained_model = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
).to("cuda")

#add token
pretrained_model.tokenizer.add_tokens(new_token)
pretrained_model.text_encoder.resize_token_embeddings(len(pretrained_model.tokenizer))

pretrained_model.save_pretrained("/kaggle/working/modified_pretrained_model")

In [ ]:
!accelerate launch /kaggle/working/diffusers/examples/dreambooth/train_dreambooth_lora.py \
  --pretrained_model_name_or_path="stable-diffusion-v1-5/stable-diffusion-v1-5" \
  --instance_data_dir="/kaggle/input/calico-cat" \
  --output_dir="/kaggle/working/dreambooth-model" \
  --instance_prompt="a photo of [V] cat" \
  --resolution=256 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=1 \
  --checkpointing_steps=100 \
  --learning_rate=1e-4 \
  --report_to="wandb" \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=200 \
  --validation_prompt="A photo of [V] cat in a bucket" \
  --validation_epochs=50 \
  --seed="0" \
  --push_to_hub

In [ ]:
# Load the Stable Diffusion model from Hugging Face
model_id = "stable-diffusion-v1-5/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id)
pipe = pipe.to("cuda")

# load the adapter layers from hugging face
pipe.unet.load_attn_procs(f"{HF_ACCOUNT}/dreambooth-model")

In [ ]:
# inference
prompt = "A photo of a [V] cat in front of Eiffel Tower"
image = pipe(prompt, num_inference_steps=100, guidance_scale=7.0).images[0]
plt.imshow(image)
plt.axis('off')  # Hide axis
plt.show()